Goto https://aistudio.google.com/app/apikey to generate the API key, then go to Add-ons to add the API key

In [ ]:
!pip uninstall -qqy jupyterlab  # Remove unused packages from Kaggle's base image that conflict
!pip install -U -q "google-genai==1.7.0"

In [ ]:
from google import genai
from google.genai import types

from IPython.display import HTML, Markdown, display

In [ ]:
from google.api_core import retry


is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

genai.models.Models.generate_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.generate_content)

In [ ]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

In [ ]:
client = genai.Client(api_key=GOOGLE_API_KEY)

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents="Explain AI to me like I'm a kid.")

print(response.text)

In [ ]:
Markdown(response.text) #render the response 

#### Start a chat

In [ ]:
chat = client.chats.create(model='gemini-2.0-flash', history=[])
response = chat.send_message('Hello! My name is Zlork.')
print(response.text)

In [ ]:
response = chat.send_message('Can you tell me something interesting about dinosaurs?')
print(response.text)

check if the model if remember (as long as chat is alive)

In [ ]:
response = chat.send_message('Do you remember what my name is?')
print(response.text)

#### Choose a model

In [ ]:
for model in client.models.list():
  print(model.name)

In [ ]:
from pprint import pprint

for model in client.models.list():
  if model.name == 'models/gemini-2.0-flash':
    pprint(model.to_json_dict())
    break

In [ ]:
{'description': 'Gemini 2.0 Flash',
 'display_name': 'Gemini 2.0 Flash',
 'input_token_limit': 1048576,
 'name': 'models/gemini-2.0-flash',
 'output_token_limit': 8192,
 'supported_actions': ['generateContent', 'countTokens'],
 'tuned_model_info': {},
 'version': '2.0'}

#### Generation parameters

- To stop the model from generating tokens past a limit, you can specify the max_output_tokens parameter when using the Gemini API.

In [ ]:
from google.genai import types

short_config = types.GenerateContentConfig(max_output_tokens=200)

response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=short_config,
    contents='Write a 1000 word essay on the importance of olives in modern society.')

print(response.text)

In [ ]:
response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=short_config,
    contents='Write a short poem on the importance of olives in modern society.')

print(response.text)

#### Temperature
- Temperature controls the degree of randomness in token selection. Higher temperatures result in a higher number of candidate tokens from which the next output token is selected, and can produce more diverse results, while lower temperatures have the opposite effect, such that a temperature of 0 results in greedy decoding, selecting the most probable token at each step.

In [ ]:
high_temp_config = types.GenerateContentConfig(temperature=2.0)


for _ in range(5):
  response = client.models.generate_content(
      model='gemini-2.0-flash',
      config=high_temp_config,
      contents='Pick a random colour... (respond in a single word)')

  if response.text:
    print(response.text, '-' * 25)

In [ ]:
Magenta
 -------------------------
Azure
 -------------------------
Azure
 -------------------------
Turquoise
 -------------------------
Cerulean
 -------------------------

In [ ]:
low_temp_config = types.GenerateContentConfig(temperature=0.0)

for _ in range(5):
  response = client.models.generate_content(
      model='gemini-2.0-flash',
      config=low_temp_config,
      contents='Pick a random colour... (respond in a single word)')

  if response.text:
    print(response.text, '-' * 25)

In [ ]:
Azure
 -------------------------
Azure
 -------------------------
Azure
 -------------------------
Azure
 -------------------------
Azure
 -------------------------

#### Top-P
- top-P parameter is also used to control the diversity of the model's output.
- Top-P defines the probability threshold that, once cumulatively exceeded, tokens stop being selected as candidates. A top-P of 0 is typically equivalent to greedy decoding, and a top-P of 1 typically selects every token in the model's vocabulary.
- You may also see top-K referenced in LLM literature. Top-K is not configurable in the Gemini 2.0 series of models, but can be changed in older models. Top-K is a positive integer that defines the number of most probable tokens from which to select the output token. A top-K of 1 selects a single token, performing greedy decoding.

In [ ]:
model_config = types.GenerateContentConfig(
    # These are the default values for gemini-2.0-flash.
    temperature=1.0,
    top_p=0.95,
    max_output_tokens=200
)

story_prompt = "You are a creative writer. Write a short story about a cat who goes on an adventure."
response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=model_config,
    contents=story_prompt)

print(response.text)

In [ ]:
Clementine, a tabby of discerning tastes and a fur coat the color of burnt caramel, was bored. Utterly, irredeemably bored. The sunbeam on the living room rug, usually a siren song of warmth and slumber, held no appeal. The feather toy, once a source of boundless joy, lay limp and forgotten. Even the tantalizing aroma of tuna emanating from the kitchen failed to pique her interest.

She needed adventure.

Clementine stretched, flexing her claws against the worn upholstery of the armchair, and leapt to the windowsill. Beyond the glass, the world hummed with a symphony of unfamiliar sounds: chirping birds, rumbling cars, the distant laughter of children. This was it. This was where adventure lurked.

The window was slightly ajar. Clementine, with the practiced ease of a seasoned escape artist, nudged it open a fraction further. The cool, crisp air kissed her whiskers, carrying the scent of damp earth and unknown flowers

In [ ]:
# top_p = 0
Clementine, a calico of discerning tastes and a perpetually unimpressed expression, considered her life utterly predictable. Sunbeam naps, chasing dust bunnies, the occasional disdainful glance at the goldfish – it was all terribly…beige. She yearned for something more, a splash of vibrant color in her monochrome existence.

One blustery autumn afternoon, the back door, usually bolted tighter than a miser's purse, was ajar. A tantalizing gust of wind, carrying the scent of damp earth and decaying leaves, beckoned. Clementine, abandoning her nap mid-yawn, slipped through the crack.

The world outside was a riot of sensory overload. Towering trees, their leaves ablaze in fiery hues, whispered secrets in the wind. Squirrels, plump and frantic, chattered insults from branches. Clementine, usually content with the confines of her sun-drenched windowsill, felt a thrill course through her. This was it. Adventure.

Her first challenge was

#### Prompting
- zero-shot are prompts that describe the request for the model directly.
- Enum mode allows you to constrain the output to a fixed set of values. (Less flexible in defining constraints, and more API level control)
- One-shot and few-shot: Providing an example of the expected response is known as a "one-shot" prompt. When you provide multiple examples, it is a "few-shot" prompt.
- JSON mode
- Chain of thought
- ReAct: Reason and act 
- Thinking mode
- Code prompting

In [ ]:
model_config = types.GenerateContentConfig(
    temperature=0.1,
    top_p=1,
    max_output_tokens=5,
)

zero_shot_prompt = """Classify movie reviews as POSITIVE, NEUTRAL or NEGATIVE.
Review: "Her" is a disturbing study revealing the direction
humanity is headed if AI is allowed to keep evolving,
unchecked. I wish there were more movies like this masterpiece.
Sentiment: """

response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=model_config,
    contents=zero_shot_prompt)

print(response.text)

In [ ]:
import enum

class Sentiment(enum.Enum):
    POSITIVE = "positive"
    NEUTRAL = "neutral"
    NEGATIVE = "negative"


response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=types.GenerateContentConfig(
        response_mime_type="text/x.enum",
        response_schema=Sentiment
    ),
    contents=zero_shot_prompt)

print(response.text)

When using constrained output like an enum, the Python SDK will attempt to convert the model's text response into a Python object automatically. It's stored in the response.parsed field.

In [ ]:
enum_response = response.parsed
print(enum_response)
print(type(enum_response))

In [ ]:
few_shot_prompt = """Parse a customer's pizza order into valid JSON:

EXAMPLE:
I want a small pizza with cheese, tomato sauce, and pepperoni.
JSON Response:
```
{
"size": "small",
"type": "normal",
"ingredients": ["cheese", "tomato sauce", "pepperoni"]
}
```

EXAMPLE:
Can I get a large pizza with tomato sauce, basil and mozzarella
JSON Response:
```
{
"size": "large",
"type": "normal",
"ingredients": ["tomato sauce", "basil", "mozzarella"]
}
```

ORDER:
"""

customer_order = "Give me a large with cheese & pineapple"

response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=types.GenerateContentConfig(
        temperature=0.1,
        top_p=1,
        max_output_tokens=250,
    ),
    contents=[few_shot_prompt, customer_order])

print(response.text)

##### JSON mode
- To provide control over the schema, and to ensure that you only receive JSON (with no other text or markdown), you can use the Gemini API's [JSON mode](https://github.com/google-gemini/cookbook/blob/main/quickstarts/JSON_mode.ipynb). This forces the model to constrain decoding, such that token selection is guided by the supplied schema.

In [ ]:
import typing_extensions as typing

class PizzaOrder(typing.TypedDict):
    size: str
    ingredients: list[str]
    type: str


response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=types.GenerateContentConfig(
        temperature=0.1,
        response_mime_type="application/json",
        response_schema=PizzaOrder,
    ),
    contents="Can I have a large dessert pizza with apple and chocolate")

print(response.text)

In [ ]:
{
  "size": "large",
  "ingredients": ["apple", "chocolate"],
  "type": "dessert"
}

##### Chain of Thought (CoT) 
- Direct prompting on LLMs can return answers quickly and (in terms of output token usage) efficiently, but they can be prone to hallucination. The answer may "look" correct (in terms of language and syntax) but is incorrect in terms of factuality and reasoning.
- Chain-of-Thought prompting is a technique where you instruct the model to output intermediate reasoning steps, and it typically gets better results, especially when combined with few-shot examples. It is worth noting that this technique doesn't completely eliminate hallucinations, and that it tends to cost more to run, due to the increased token count.

In [ ]:
prompt = """When I was 4 years old, my partner was 3 times my age. Now, I
am 20 years old. How old is my partner? Return the answer directly."""

response = client.models.generate_content(
    model='gemini-2.0-flash',
    contents=prompt)

print(response.text)

In [ ]:
52

In [ ]:
prompt = """When I was 4 years old, my partner was 3 times my age. Now,
I am 20 years old. How old is my partner? Let's think step by step."""

response = client.models.generate_content(
    model='gemini-2.0-flash',
    contents=prompt)

Markdown(response.text)

In [ ]:
Here's how to solve the problem step-by-step:

Find the age difference: When you were 4, your partner was 3 times your age, meaning they were 4 * 3 = 12 years old.
Calculate the age difference: The age difference between you and your partner is 12 - 4 = 8 years.
Apply the age difference to your current age: Since your partner is 8 years older than you, they are currently 20 + 8 = 28 years old.
Answer: Your partner is now 28 years old.

##### ReAct: Reason and act 
- In this example you will run a ReAct prompt directly in the Gemini API and perform the searching steps yourself. As this prompt follows a well-defined structure, there are frameworks available that wrap the prompt into easier-to-use APIs that make tool calls automatically, such as the LangChain example from the "Prompting" whitepaper.
- To try this out with the Wikipedia search engine, check out the Searching Wikipedia with ReAct cookbook example.

In [ ]:
model_instructions = """
Solve a question answering task with interleaving Thought, Action, Observation steps. Thought can reason about the current situation,
Observation is understanding relevant information from an Action's output and Action can be one of three types:
 (1) <search>entity</search>, which searches the exact entity on Wikipedia and returns the first paragraph if it exists. If not, it
     will return some similar entities to search and you can try to search the information from those topics.
 (2) <lookup>keyword</lookup>, which returns the next sentence containing keyword in the current context. This only does exact matches,
     so keep your searches short.
 (3) <finish>answer</finish>, which returns the answer and finishes the task.
"""

example1 = """Question
Musician and satirist Allie Goertz wrote a song about the "The Simpsons" character Milhouse, who Matt Groening named after who?

Thought 1
The question simplifies to "The Simpsons" character Milhouse is named after who. I only need to search Milhouse and find who it is named after.

Action 1
<search>Milhouse</search>

Observation 1
Milhouse Mussolini Van Houten is a recurring character in the Fox animated television series The Simpsons voiced by Pamela Hayden and created by Matt Groening.

Thought 2
The paragraph does not tell who Milhouse is named after, maybe I can look up "named after".

Action 2
<lookup>named after</lookup>

Observation 2
Milhouse was named after U.S. president Richard Nixon, whose middle name was Milhous.

Thought 3
Milhouse was named after U.S. president Richard Nixon, so the answer is Richard Nixon.

Action 3
<finish>Richard Nixon</finish>
"""

example2 = """Question
What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?

Thought 1
I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.

Action 1
<search>Colorado orogeny</search>

Observation 1
The Colorado orogeny was an episode of mountain building (an orogeny) in Colorado and surrounding areas.

Thought 2
It does not mention the eastern sector. So I need to look up eastern sector.

Action 2
<lookup>eastern sector</lookup>

Observation 2
The eastern sector extends into the High Plains and is called the Central Plains orogeny.

Thought 3
The eastern sector of Colorado orogeny extends into the High Plains. So I need to search High Plains and find its elevation range.

Action 3
<search>High Plains</search>

Observation 3
High Plains refers to one of two distinct land regions

Thought 4
I need to instead search High Plains (United States).

Action 4
<search>High Plains (United States)</search>

Observation 4
The High Plains are a subregion of the Great Plains. From east to west, the High Plains rise in elevation from around 1,800 to 7,000 ft (550 to 2,130m).

Thought 5
High Plains rise in elevation from around 1,800 to 7,000 ft, so the answer is 1,800 to 7,000 ft.

Action 5
<finish>1,800 to 7,000 ft</finish>
"""

# Come up with more examples yourself, or take a look through https://github.com/ysymyth/ReAct/

In [ ]:
question = """Question
Who was the youngest author listed on the transformers NLP paper?
"""

# You will perform the Action; so generate up to, but not including, the Observation.
react_config = types.GenerateContentConfig(
    stop_sequences=["\nObservation"],
    system_instruction=model_instructions + example1 + example2,
)

# Create a chat that has the model instructions and examples pre-seeded.
react_chat = client.chats.create(
    model='gemini-2.0-flash',
    config=react_config,
)

resp = react_chat.send_message(question)
print(resp.text)

In [ ]:
Thought 1
I need to find the transformers NLP paper, then identify all the authors, and determine the youngest author.

Action 1
<search>transformers NLP paper</search>

In [ ]:
# Now you can perform this research yourself and supply it back to the model.
observation = """Observation 1
[1706.03762] Attention Is All You Need
Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely.
"""
resp = react_chat.send_message(observation)
print(resp.text)

In [ ]:
Thought 2
The authors listed on the paper are: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin. I need to find the age of each author and find the youngest.

Action 2
<search>Ashish Vaswani age</search>

##### Thinking mode
- The experiemental Gemini Flash 2.0 "Thinking" model has been trained to generate the "thinking process" the model goes through as part of its response. As a result, the Flash Thinking model is capable of stronger reasoning capabilities in its responses.
- Using a "thinking mode" model can provide you with high-quality responses without needing specialised prompting like the previous approaches. One reason this technique is effective is that you induce the model to generate relevant information ("brainstorming", or "thoughts") that is then used as part of the context in which the final response is generated.
- Note that when you use the API, you get the final response from the model, but the thoughts are not captured. To see the intermediate thoughts, try out the thinking mode model in AI Studio. 

In [ ]:
import io
from IPython.display import Markdown, clear_output


response = client.models.generate_content_stream(
    model='gemini-2.0-flash-thinking-exp',
    contents='Who was the youngest author listed on the transformers NLP paper?',
)

buf = io.StringIO()
for chunk in response:
    buf.write(chunk.text)
    # Display the response as it is streamed
    print(chunk.text, end='')

# And then render the finished response as formatted markdown.
clear_output()
Markdown(buf.getvalue())

##### Code prompting

In [ ]:
# The Gemini models love to talk, so it helps to specify they stick to the code if that
# is all that you want.
code_prompt = """
Write a Python function to calculate the factorial of a number. No explanation, provide only the code.
"""

response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=types.GenerateContentConfig(
        temperature=1,
        top_p=1,
        max_output_tokens=1024,
    ),
    contents=code_prompt)

Markdown(response.text)

In [ ]:
# can also execute code 
from pprint import pprint

config = types.GenerateContentConfig(
    tools=[types.Tool(code_execution=types.ToolCodeExecution())],
)

code_exec_prompt = """
Generate the first 14 odd prime numbers, then calculate their sum.
"""

response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=config,
    contents=code_exec_prompt)

for part in response.candidates[0].content.parts:
  pprint(part.to_json_dict())
  print("-----")

In [ ]:
{'text': "Okay, I can do that. First, I'll generate the first 14 odd prime "
         'numbers. Remember that a prime number is a number greater than 1 '
         'that has no positive divisors other than 1 and itself. Also, since '
         "the question asks for *odd* prime numbers, I'll exclude 2, which is "
         'the only even prime number.\n'
         '\n'
         'The first few prime numbers are 2, 3, 5, 7, 11, 13, 17, 19, 23, 29, '
         '31, 37, 41, 43, 47, 53.  Excluding 2 and taking the next 14, we get: '
         '3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47.\n'
         '\n'
         "Now I'll calculate the sum of these numbers using a python tool.\n"
         '\n'}
-----
{'executable_code': {'code': 'numbers = [3, 5, 7, 11, 13, 17, 19, 23, 29, 31, '
                             '37, 41, 43, 47]\n'
                             'total = sum(numbers)\n'
                             'print(total)\n',
                     'language': 'PYTHON'}}
-----
{'code_execution_result': {'outcome': 'OUTCOME_OK', 'output': '326\n'}}
-----
{'text': 'The first 14 odd prime numbers are 3, 5, 7, 11, 13, 17, 19, 23, 29, '
         '31, 37, 41, 43, and 47. Their sum is 326.\n'}
-----


In [ ]:
for part in response.candidates[0].content.parts:
    if part.text:
        display(Markdown(part.text))
    elif part.executable_code:
        display(Markdown(f'```python\n{part.executable_code.code}\n```'))
    elif part.code_execution_result:
        if part.code_execution_result.outcome != 'OUTCOME_OK':
            display(Markdown(f'## Status {part.code_execution_result.outcome}'))

        display(Markdown(f'```\n{part.code_execution_result.output}\n```'))

In [ ]:
Okay, I can do that. First, I'll generate the first 14 odd prime numbers. Remember that a prime number is a number greater than 1 that has no positive divisors other than 1 and itself. Also, since the question asks for odd prime numbers, I'll exclude 2, which is the only even prime number.

The first few prime numbers are 2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53. Excluding 2 and taking the next 14, we get: 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47.

Now I'll calculate the sum of these numbers using a python tool.

numbers = [3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47]
total = sum(numbers)
print(total)
326
The first 14 odd prime numbers are 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, and 47. Their sum is 326.

In [ ]:
# explain code
file_contents = !curl https://raw.githubusercontent.com/magicmonty/bash-git-prompt/refs/heads/master/gitprompt.sh

explain_prompt = f"""
Please explain what this file does at a very high level. What is it, and why would I use it?

```
{file_contents}
```
"""

response = client.models.generate_content(
    model='gemini-2.0-flash',
    contents=explain_prompt)

Markdown(response.text)

#### Learn more
- Check out the whitepaper issued with today's content,
- Try out the apps listed at the top of this notebook (TextFX, SQL Talk and NotebookLM),
- Read the Introduction to Prompting from the Gemini API docs,
- Explore the Gemini API's prompt gallery and try them out in AI Studio,
- Check out the Gemini API cookbook for inspirational examples and educational quickstarts.
- Be sure to check out the codelabs on day 3 too, where you will explore some more advanced prompting with code execution.